## Librerias

In [1]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## SAP2000

In [ ]:
AttachToInstance = False
SpecifyPath = True
ProgramPath = "C:\Program Files\Computers and Structures\SAP2000 22\SAP2000.exe"
APIPath = r'D:\Users\User(D)\Documentos\GitHub\freeproyect\SAP2000\Modelos'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'portico.sdb'
helper = comtypes.client.CreateObject('SAP2000v1.Helper')
helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
if AttachToInstance:
    try:
        mySapObject = helper.GetObject("CSI.SAP2000.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            mySapObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    mySapObject.ApplicationStart()
SapModel = mySapObject.SapModel
SapModel.InitializeNewModel()

## Modo de conexion con el programa
+ Abriendo automaticamente una ventana de Etabas
+ Ingresando informacion a traves de codigo python

In [2]:
#True:Trabaja con el modelo existente
#False:Crea un nuevo modelo
AttachToInstance = False

## Complemento de conexion

In [3]:
#True:especifica manualmente la ruata a ETABS.exe
#False:se usara la ultima version instalada de Etabs
SpecifyPath = True

#Si el indicador anterior esta en ruta, especifica la ruta a ETABS a continuacion
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 19\ETABS.exe"

#ruta completa del modelo
#ajuste a la ruta deseada de su modelo
APIPath =r'D:\Users\User(D)\Escritorio\freeproyect\Etabs'

if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'API_1-001.edb'

#Crea un objeto api de ayuda
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

if AttachToInstance:
    #attach to a running instance of ETABS
    try:
        #get the active ETABS object
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            #'create an instance of the ETABS object from the specified path
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            #create an instance of the ETABS object from the latest installed ETABS
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    #Comezar la aplicacion de ETABS
    myETABSObject.ApplicationStart()

#create SapModel object
SapModel = myETABSObject.SapModel

#initialize model
SapModel.InitializeNewModel()

## Definicion de espacio de trabajo
+ NewBlank()
+ NewGriddOnly()
ret = SapModel.File.NewGridOnly(10,4,3,3,3,2,2)
+ NewSteelDeck()
ret = SapModel.File.NewSteelDeck(5,4,3,4,4,24,24)
+ OpenFile()
PathFile=""
ret = SapModel.File.OpenFile(PathFile)

In [ ]:
SapModel.File.NewBlank()

In [4]:
SapModel.File.NewGridOnly(10,4,3,3,3,24,24)

0

## Definimos propiedad de material
+ Steel 1
+ Concreto 2
+ Acero de refuerzo 6
+ Albañileria 8

In [5]:
#define material property
MATERIAL_CONCRETE = 2   
ret=SapModel.PropMaterial.SetMaterial('CONCRETO', 
                                        MATERIAL_CONCRETE)

#assign isotropic mechanical properties to material
ret=SapModel.PropMaterial.SetMPIsotropic('CONCRETO', 
                                           3600, #elasticidad
                                           0.2,  #poison
                                           0.0000055)#coef.termico

0

In [8]:
#define material property
REFUERZO = 6  
ret=SapModel.PropMaterial.SetMaterial('REFUERZO', 
                                        REFUERZO)

#assign isotropic mechanical properties to material
ret=SapModel.PropMaterial.SetMPIsotropic('REFUERZO', 
                                        1234, #elasticidad
                                        0.2,  #poison
                                        0.0000055)#coef.termico

In [9]:
ret=SapModel.PropMaterial.SetWeightAndMass('CONCRETO',1,280)

In [10]:
ret=SapModel.PropMaterial.SetOConcrete('CONCRETO',5,False, 0, 1, 2, 0.0022, 0.0052)

In [ ]:
ret = SapModel.PropMaterial.SetORebar("Rebar", 62, 93, 70, 102, 2, 2, 0.02, 0.1, False)

In [12]:
#define material property
MAETERIAL_MASONRY = 8
ret=Sret=SapModel.PropMaterial.SetMaterial('ALBAÑILERIA', 
                                        MAETERIAL_MASONRY)

#assign isotropic mechanical properties to material
ret=SapModel.PropMaterial.SetMPIsotropic('ALBAÑILERIA', 
                                        3600, #elasticidad
                                        0.2,  #poison
                                        0.0000055)#coef.termico

## Definimos unidades
+ kip_ft_F = 4
+ kg_m_C=8
+ tonf_m_C=12

In [13]:
#switch to k-ft units
kg_m_C=8
ret = SapModel.SetPresentUnits(kg_m_C)

## Definimos frames

In [14]:

#define rectangular frame section property
ret = SapModel.PropFrame.SetRectangle('R1', #Name
                                      'CONCRETO', #Material
                                      12, #peralte
                                      12) #base

In [15]:
ret = SapModel.PropFrame.SetRebarBeam('R1',             #nombre de frame existente
                                    'REFUERZO',         #Material de acero longitudinal
                                    'REFUERZO',         #Material de acero transversal
                                    3.5,                #Recubrimiento superior
                                    3,                  #Recubrimiento inferior
                                    4.1,                #Top left area
                                    4.2,                #Top Right area
                                    4.3,                #Bottom left area
                                    4.4)                #Bottom right area

In [17]:
ret = SapModel.PropFrame.SetRebarColumn("R2",           #Nombre de un frame existente
                                        'REFUERZO',      #Material del acero longitudinal
                                        'REFUERZO',      #Material del acero transversal
                                        2,              #Patter(rectangular=1/circular=2)
                                        2,              #Tipo de confinamiento(ties=1/espiral=2)
                                        2,              #Recubrimiento de columna cm
                                        10,             #solo aplica para columna circular, numero total de barras
                                        0,              #solo aplica para la columna rectangular,numero de barras en el eje local 3
                                        0,              #solo aplica para la columna rectangular,numero de barras en el eje local 1
                                        "#10",          #Diametro del acero longitudinal
                                        "#5",           #Diametro del acero transversal
                                        4,              #espaciamiento de estribos
                                        0,              #numero de estribos alrededor del eje local 2
                                        0,              #numero de estribos alrededor del eje local 3
                                        False)

In [18]:

#define frame section property modifiers
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('R1', ModValue)

## Slab
Slab Type
+ Slab=0
+ Drop=1

Shell Type
+ ShellThin=1
+ ShellThick=2

In [19]:
ret = SapModel.PropArea.SetSlab('Losa maciza',#Name
                                0,#Slab Type
                                1,#Shell Type
                                'CONCRETO',#Material
                                15)#Thickness

In [ ]:
ret = SapModel.PropArea.SetSlab('Losa maciza',0,1,'CONCRETO',15)

## Wall
Wall Type
+ Specified=1
+ AutoSelectList=2

Shell Type
+ ShellThin=1
+ ShellThick=2
+ Membrane=3
+ Layered=6

In [ ]:
ret = SapModel.PropArea.SetWall('MURO ALBAÑILERIA', #Name
                                1, #Wall Type
                                1,  #Shell Type
                                'ALBAÑILERIA',      #Material
                                14.2)           #Thickness

In [ ]:
ret = SapModel.PropArea.SetWall('MURO ALBAÑILERIA', 1,1, 'ALBAÑILERIA',14.2) 

## Diaphragm

In [ ]:
#Creacion de diaphragm
ret=SapModel.Diaphragm.SetDiaphragm('D1',#name
                            SemiRigid=True),#booleano

# obtenemos nombre y etiqueta de los diaphragm
ret=SapModel.AreaObj.GetLabelNameList('numer name',# nombre de numero
                                      'D1',#my nombre
                                      'label',#my etiqueta
                                      'story'),#my story

# asignacion de diaphragm
ret=SapModel.cAreaObj.SetDiaphragm('D1',#name
                                   'D1')#DiaphragmName 

### Agregamos Frame

In [ ]:

#add frame object by coordinates
FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'R1', '1', 'Global')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1', '2', 'Global')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1', '3', 'Global')

In [ ]:
ret=SapModel.Diaphragm.SetDiaphragm('D1',SemiRigid=True)
ret=SapModel.AreaObj.GetLabelNameList('numer name','D1','label','story')

## Agregamos Wall

In [ ]:
ret = SapModel.AreaObj.AddByCoord(6, #numero de puntos
                                  x, #coordenas lista x
                                  y, #coordenas lista y
                                  z, #coordenas lista z
                                  "muro") #nombre

In [ ]:
ret = SapModel.AreaObj.AddByCoord(6, x, y, z, Name)

## Load patterns
LoadPatternType
+ Dead=1
+ SuperDead=2
+ Live=3
+ ReduceLive=4
+ Quake=5
+ Wind=6
+ Snow=7
+ Other=8
+ Move=9
+ Temperature=10
+ Rooflive=11

In [ ]:

#add load patterns
ret = SapModel.LoadPatterns.Add('Dead',     #Name
                                1,          #LoadPatternType 
                                1,          #Self weight multiplier           
                                True)       #Analis caso

ret = SapModel.LoadPatterns.Add('SuperDead', 2, 0, True)
ret = SapModel.LoadPatterns.Add('Live', 3, 0, True)
ret = SapModel.LoadPatterns.Add('ReduceLive', 4, 0, True)
ret = SapModel.LoadPatterns.Add('Quake', 5, 0, True)
ret = SapModel.LoadPatterns.Add('Wind', 6, 0, True)
ret = SapModel.LoadPatterns.Add('Snow', 7, 0, True)

## Carga Puntual

In [ ]:

#assign loading for load pattern
PointLoadValue = [0,0,-10,0,0,0]
ret = SapModel.PointObj.SetLoadForce("punto 1",'Dead',PointLoadValue)
ret = SapModel.PointObj.SetLoadForce("punto 1", #Name
                                    'Dead',           #LoadPattern
                                    PointLoadValue) #point
ret = SapModel.PointObj.SetLoadForce("punto 1",'Dead',PointLoadValue)


In [ ]:
Pointi=""
Pointj=""
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, Pointi, Pointj)
PointLoadValue = [0,0,-10,0,0,0]
ret = SapModel.PointObj.SetLoadForce("punto 1",'Dead',PointLoadValue)

## Carga Distribuida
+ 1.Local 1 axis (only applies when CSys is Local)
+ 2.Local 2 axis (only applies when CSys is Local)
+ 3.Local 3 axis (only applies when CSys is Local)
+ 4.X direction (does not apply when CSys is Local)
+ 5.Y direction (does not apply when CSys is Local)
+ 6.Z direction (does not apply when CSys is Local)
+ 7.Projected X direction (does not apply when CSys is Local)
+ 8.Projected Y direction (does not apply when CSys is Local)
+ 9.Projected Z direction (does not apply when CSys is Local)
+ 10.Gravity direction (only applies when CSys is Global)
+ 11.Projected Gravity direction (only applies when CSys is Global)

In [ ]:
ret = SapModel.FrameObj.SetLoadDistributed(FrameName3, #Name
                                           '2', #LoadPattern
                                           1, #Type of load distribution 1:F/L 2:M/L
                                           10, #Direccion de Load Distribution
                                           0, #Distancia of start of load distribution(relative por default)
                                           1, #Distancia of end of load distribution(relative por default)
                                           1.8, #Value of start of load distribution
                                           1.8,
                                           "Global")#Value of end of load distribution
    #existen mas parametros a modificar

In [ ]:
ret = SapModel.FrameObj.SetLoadDistributed(FrameName3,'2',1,10,0,1,1.8,1.8)

In [ ]:

#assign loading for load pattern 3
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0,0,-17.2,0,-54.4,0]
ret = SapModel.PointObj.SetLoadForce(PointName2, '3', PointLoadValue)

#assign loading for load pattern 4
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '4', 1, 11, 0, 1, 2, 2)

#assign loading for load pattern 5
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '5', 1, 2, 0, 1, 2, 2, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '5', 1, 2, 0, 1, -2, -2, 'Local')

#assign loading for load pattern 6
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '6', 1, 2, 0, 1, 0.9984, 0.3744, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '6', 1, 2, 0, 1, -0.3744, 0, 'Local')

#assign loading for load pattern 7
ret = SapModel.FrameObj.SetLoadPoint(FrameName2, '7', 1, 2, 0.5, -15, 'Local')

## Definimos restricciones

In [ ]:

#assign point object restraint at base
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, True, False, False]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

#assign point object restraint at top
Restraint = [True, True, False, False, False, False]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName2, Restraint)

## Actualizamos el modelo

In [13]:

#refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

## Guardamos el modelo

In [ ]:
#save model
os.makedirs(r"D:\Users\User(D)\Escritorio\freeproyect\Etabs\prueba",exist_ok=True)
ret = SapModel.File.Save(r'D:\Users\User(D)\Escritorio\freeproyect\Etabs\prueba\ejemplo.edb')

## Analizamos el modelo

In [ ]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

## Extraemos informacion

In [ ]:
#initialize for results
ProgramResult = [0,0,0,0,0,0,0]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)

#get results for load cases 1 through 7
for i in range(0,7):
      NumberResults = 0
      Obj = []
      Elm = []
      ACase = []
      StepType = []
      StepNum = []
      U1 = []
      U2 = []
      U3 = []
      R1 = []
      R2 = []
      R3 = []
      ObjectElm = 0
      ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
      ret = SapModel.Results.Setup.SetCaseSelectedForOutput(str(i + 1))
      if i <= 3:
          [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = SapModel.Results.JointDispl(PointName2, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
          ProgramResult[i] = U3[0]
      else:
          [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = SapModel.Results.JointDispl(PointName1, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
          ProgramResult[i] = U1[0]

In [ ]:

#close the program
ret = myETABSObject.ApplicationExit(False)
SapModel = None
myETABSObject = None

In [ ]:

#fill independent results
IndResult = [0,0,0,0,0,0,0]
IndResult[0] = -0.02639
IndResult[1] = 0.06296
IndResult[2] = 0.06296
IndResult[3] = -0.2963
IndResult[4] = 0.3125
IndResult[5] = 0.11556
IndResult[6] = 0.00651

#fill percent difference
PercentDiff = [0,0,0,0,0,0,0]
for i in range(0,7):
      PercentDiff[i] = (ProgramResult[i] / IndResult[i]) - 1

#display results
for i in range(0,7):
      print()
      print(ProgramResult[i])
      print(IndResult[i])
      print(PercentDiff[i])